In [28]:
import numpy as np
import pandas as pd
import re,math

name = "410365"
# 读取空columns的以第一列为行索引的除去最后一列第122列的数据
data = pd.read_csv(f'{name}.csv',index_col = 0, header = None).drop(columns = [122])

In [29]:
data

,1,2,3,4,5,6,7,8,9,10,...,112,113,114,115,116,117,118,119,120,121
0,,,,,,,,,,,,,,,,,,,,,
1.png,2.601425,5.026990,2.701762,3.879969,0.898453,1.308082,0.916988,1.123952,0.691883,0.619557,...,2.569651,160.050877,106.879498,153.927052,53.041939,59.528150,41.390650,-37.951233,48.441674,10.916512
10.png,3.129841,4.288597,2.725429,5.337885,0.991238,1.172903,0.908364,1.335855,0.674477,0.644072,...,2.669664,160.015675,104.870255,151.986095,51.488186,50.930881,35.097160,-47.648261,25.582701,-33.068379
100.png,2.575046,4.577409,1.958165,3.202430,0.858012,1.183237,0.721216,0.954617,0.709781,0.653841,...,2.388612,160.416473,104.851959,151.978241,51.350462,50.699006,35.491655,-48.220212,31.804248,-31.955316
1000.png,2.664400,4.160769,2.311428,4.221899,0.928431,1.202707,0.850429,1.193464,0.681145,0.629362,...,2.600121,159.204521,106.530697,153.800426,53.613169,57.373312,38.534735,-40.313692,42.531263,-17.950770
1001.png,2.440472,3.786251,2.209084,3.973103,0.876961,1.126905,0.833747,1.164860,0.692097,0.643791,...,2.536722,161.158566,105.148163,152.088215,49.873871,52.348003,37.910457,-46.304806,37.323233,-32.399455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994.png,2.962102,4.739900,2.429841,4.258608,0.988358,1.286028,0.876685,1.206890,0.667151,0.615226,...,2.659943,160.095455,106.545052,153.662064,51.912590,55.937589,38.665904,-41.556536,41.311903,-20.350152
995.png,2.447718,4.011057,2.443612,4.244198,0.875965,1.159016,0.874433,1.203614,0.693766,0.640201,...,2.594307,161.136787,106.551125,153.702164,51.235749,57.133972,39.087126,-40.499179,45.354599,-16.919474
996.png,3.855499,6.037078,3.217831,5.662284,1.141835,1.477263,1.035080,1.431634,0.642039,0.588570,...,2.854436,161.582607,106.376381,153.386810,51.194369,58.219918,40.453625,-40.349174,47.007056,-15.096018


In [30]:
# 多线程方差求解
import threading
import queue
from tqdm import tqdm

q = queue.Queue()
result_q = queue.Queue()

for elem in range(1,len(data.columns)+1):
    q.put(elem)

mean_list = []
for i in data.columns:
    mean_list.append(data[i].mean())

def temp_f():
    while True:
        if q.qsize() != 0:
            elem = q.get()
            tp = []
            mean = mean_list[elem-1]
            for j in list(data[elem]):
                ttpp = np.square(j - mean)
                tp.append(ttpp)
            result_q.put((elem, tp))
        else:
            break

threads = []
for i in tqdm(range(len(data.columns))):
    t = threading.Thread(target=temp_f)
    threads.append(t)
    t.start()

print('主程序运行中...')

# 等待所有线程任务结束。
for t in threads:
    t.join()

print("所有线程任务完成")

100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [00:01<00:00, 69.45it/s]

主程序运行中...
所有线程任务完成


In [31]:
# 栈反解数据列表
result_list = []
for i in range(result_q.qsize()):
    j = result_q.get()
    result_list.append(j)
result_list.sort(key=lambda x:x[0]) # 乱序排序

In [32]:
# 生成方差dataframe  data2
data2_list = []
for i in result_list:
    data2_list.append(i[1])
data2 = pd.DataFrame(data2_list,columns=data.index,index=data.columns).T
data2

,1,2,3,4,5,6,7,8,9,10,...,112,113,114,115,116,117,118,119,120,121
0,,,,,,,,,,,,,,,,,,,,,
1.png,0.004474,0.552221,0.094796,0.146096,0.000002,0.017293,0.005335,0.002357,4.606871e-06,6.106311e-04,...,0.000321,0.075168,0.638841,0.789043,1.314032,7.308980,3.427131,9.571297,77.021229,1276.990751
10.png,0.213009,0.000022,0.109930,1.157113,0.008370,0.000014,0.004150,0.026683,3.822952e-04,3.850068e-08,...,0.013905,0.095710,1.464022,1.108125,0.166008,34.736364,19.733492,43.603274,198.325238,68.060579
100.png,0.008698,0.086163,0.189841,1.123099,0.001742,0.000044,0.015062,0.047475,2.481016e-04,9.163403e-05,...,0.026613,0.008359,1.508630,1.124723,0.297205,37.523361,16.384235,51.483906,61.799283,50.934219
1000.png,0.000015,0.015155,0.006797,0.001624,0.000823,0.000683,0.000042,0.000439,1.660097e-04,2.222017e-04,...,0.002340,1.255573,0.202927,0.580120,2.949952,0.301044,1.009350,0.534788,8.212601,47.165646
1001.png,0.051911,0.247628,0.034147,0.083574,0.000519,0.002467,0.000104,0.000058,3.735758e-06,2.276681e-07,...,0.000226,0.694757,0.868735,0.903556,4.087495,20.040256,2.653451,27.665734,5.486216,57.470953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994.png,0.086313,0.207960,0.001294,0.000013,0.007852,0.011979,0.001072,0.001182,7.224770e-04,8.434493e-04,...,0.011707,0.052711,0.216066,0.388494,0.000288,0.786855,0.762994,0.261685,2.710644,19.966091
995.png,0.048662,0.074429,0.002474,0.000324,0.000566,0.000308,0.000930,0.000968,6.948535e-08,1.654524e-05,...,0.001811,0.658926,0.221749,0.440091,0.435438,0.095688,0.204551,0.297903,32.365853,62.394536
996.png,1.409415,3.073726,0.678907,1.960252,0.058606,0.090411,0.036533,0.067148,2.703011e-03,3.102324e-03,...,0.091621,1.581463,0.087710,0.121132,0.491762,1.946809,0.835809,0.484151,53.898452,94.526560


In [33]:
# for i,j in data2.iterrows():
#     p = sum( k > j.mean() for k in j)
#     if p > 61:
#         print("Y")

In [34]:
# 多线程判断求解
import threading
import queue
from tqdm import tqdm

g = queue.Queue()
result_g = queue.Queue()

for elem in range(len(data2.index)):
    g.put(elem)

rows_list = []
for i,j in data2.iterrows():
    rows_list.append((i,j))

def temp_g():
    while True:
        if g.qsize() != 0:
            elem = g.get()
            yuzhi = rows_list[elem][1].mean()
            p = sum( i > yuzhi for i in rows_list[elem][1])
            result_g.put((elem, p))
        else:
            break

threads = []
for i in tqdm(range(len(data2.index))):
    t = threading.Thread(target=temp_g)
    threads.append(t)
    t.start()

print('主程序运行中...')

# 等待所有线程任务结束。
for t in threads:
    t.join()

print("所有线程任务完成")

# 栈反解数据列表
result2_list = []
for i in range(result_g.qsize()):
    j = result_g.get()
    result2_list.append(j)
result2_list.sort(key=lambda x:x[0]) # 乱序排序

#result2_list

100%|█████████████████████████████████████████████████████████████████████████████| 2310/2310 [00:03<00:00, 720.26it/s]


主程序运行中...
所有线程任务完成


In [35]:
drop_name_list = []
for i in result2_list:
    if i[1] > 15:
        # print(data2.index[i[0]])
        drop_name_list.append(data2.index[i[0]])
data3 = data.drop(drop_name_list)

In [38]:
data3

,1,2,3,4,5,6,7,8,9,10,...,112,113,114,115,116,117,118,119,120,121
0,,,,,,,,,,,,,,,,,,,,,
1.png,2.601425,5.026990,2.701762,3.879969,0.898453,1.308082,0.916988,1.123952,0.691883,0.619557,...,2.569651,160.050877,106.879498,153.927052,53.041939,59.528150,41.390650,-37.951233,48.441674,10.916512
10.png,3.129841,4.288597,2.725429,5.337885,0.991238,1.172903,0.908364,1.335855,0.674477,0.644072,...,2.669664,160.015675,104.870255,151.986095,51.488186,50.930881,35.097160,-47.648261,25.582701,-33.068379
100.png,2.575046,4.577409,1.958165,3.202430,0.858012,1.183237,0.721216,0.954617,0.709781,0.653841,...,2.388612,160.416473,104.851959,151.978241,51.350462,50.699006,35.491655,-48.220212,31.804248,-31.955316
1000.png,2.664400,4.160769,2.311428,4.221899,0.928431,1.202707,0.850429,1.193464,0.681145,0.629362,...,2.600121,159.204521,106.530697,153.800426,53.613169,57.373312,38.534735,-40.313692,42.531263,-17.950770
1001.png,2.440472,3.786251,2.209084,3.973103,0.876961,1.126905,0.833747,1.164860,0.692097,0.643791,...,2.536722,161.158566,105.148163,152.088215,49.873871,52.348003,37.910457,-46.304806,37.323233,-32.399455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994.png,2.962102,4.739900,2.429841,4.258608,0.988358,1.286028,0.876685,1.206890,0.667151,0.615226,...,2.659943,160.095455,106.545052,153.662064,51.912590,55.937589,38.665904,-41.556536,41.311903,-20.350152
995.png,2.447718,4.011057,2.443612,4.244198,0.875965,1.159016,0.874433,1.203614,0.693766,0.640201,...,2.594307,161.136787,106.551125,153.702164,51.235749,57.133972,39.087126,-40.499179,45.354599,-16.919474
996.png,3.855499,6.037078,3.217831,5.662284,1.141835,1.477263,1.035080,1.431634,0.642039,0.588570,...,2.854436,161.582607,106.376381,153.386810,51.194369,58.219918,40.453625,-40.349174,47.007056,-15.096018


In [37]:
data3.to_csv(f"{name}_drop_by_variance.csv",header = None)

In [ ]:
# tp_list = []
# for i in data.columns:
#     mean = data[i].mean()
#     tp = []
#     for j in list(data[i]):
#         ttpp = np.square(j - mean)
#         tp.append(ttpp)
#     tp_list.append(tp)
# pd.DataFrame(tp_list)